In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
movie_plots_path = "/content/drive/My Drive/Practical Introduction to NLP Course/Module1/American_Movie_Plots_2005_2021_v1.csv"

In [ ]:
import pandas as pd 
df = pd.read_csv(movie_plots_path) 
print (df.shape)
df.head()

(3409, 4)


,Unnamed: 0,Movie Name,Wiki Link,Plot
0,0,White Noise,https://en.wikipedia.org//wiki/White_Noise_(20...,Jonathan Rivers is an architect and lives with...
1,1,Coach Carter,https://en.wikipedia.org//wiki/Coach_Carter,"Ken Carter lives in Richmond, California. He b..."
2,2,Elektra,https://en.wikipedia.org//wiki/Elektra_(2005_f...,"After being killed in Daredevil, Elektra Natch..."
3,3,Racing Stripes,https://en.wikipedia.org//wiki/Racing_Stripes,"During a thunderstorm, a traveling circus, Cir..."
4,4,Tom and Jerry: Blast Off to Mars,https://en.wikipedia.org//wiki/Tom_and_Jerry:_...,Tom (voiced by Bill Kopp) chases Jerry as usua...


In [ ]:
print (df.shape)
df.drop_duplicates(subset ="Wiki Link", keep = False, inplace = True)
print (df.shape)

(3409, 4)
(3379, 4)


In [ ]:
movie_df  = df[df['Movie Name'].str.match('Interstellar')]
movie_plot = movie_df.iloc[0]['Plot']
print (movie_plot)

In 2067, crop blights and dust storms threaten humanity's survival. Cooper, a widowed engineer and former NASA pilot turned farmer, lives with his father-in-law, Donald, teen son, Tom, and 10-year-old daughter, Murph. After a dust storm, patterns inexplicably appear on Murph's dust-covered bedroom floor. Murph believes a ghost created them, but Cooper deduces the patterns were made by gravity variations and represent geographic coordinates in binary code. Cooper follows the coordinates to a secret NASA facility headed by Professor John Brand, whom Cooper knows.

Professor Brand reveals that, forty-eight years earlier, a group of unknown, presumingly extra-terrestrial beings placed a wormhole near Saturn, opening a path to a distant galaxy with twelve potentially habitable worlds located near a supermassive black hole named Gargantua. Twelve volunteers traveled through the wormhole to survey the planets, and three — Dr. Mann, Laura Miller, and Wolf Edmunds — reported positive results. P

# Extract Keywords

In [ ]:
!python3 -m spacy download en_core_web_md

2022-09-05 10:10:34.244843: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.8 MB 1.8 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


# Restart runtime after spacy download

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md')


def get_noun_phrases(sample_text):
  candidates =[]
  doc = nlp(sample_text)
  for np in doc.noun_chunks:
    phrase = np.text
    # no_of_words = phrase.split()
    # if len(no_of_words)>=2 and len(no_of_words)<=4:
    #   phrase = np.text
    phrase = phrase.strip()
    if phrase not in candidates:
      candidates.append(phrase)
  return candidates

print (get_noun_phrases(movie_plot))

['crop blights', 'dust storms', "humanity's survival", 'Cooper', 'a widowed engineer', 'former NASA pilot', 'farmer', 'his father', 'law', 'Donald', 'teen son', 'Tom', '10-year-old daughter', 'Murph', 'a dust storm', 'patterns', "Murph's dust-covered bedroom floor", 'a ghost', 'them', 'the patterns', 'gravity variations', 'geographic coordinates', 'binary code', 'the coordinates', 'a secret NASA facility', 'Professor John Brand', 'whom', 'Professor Brand', 'a group', 'unknown, presumingly extra-terrestrial beings', 'a wormhole', 'Saturn', 'a path', 'a distant galaxy', 'twelve potentially habitable worlds', 'a supermassive black hole', 'Gargantua', 'Twelve volunteers', 'the wormhole', 'the planets', 'Dr. Mann', 'Laura Miller', 'Wolf Edmunds', 'positive results', 'two plans', 'Plan A', 'an antigravitational propulsion theory', 'settlements', 'space', 'contrast', 'Plan B', 'the Endurance spacecraft', '5,000 frozen human embryos', 'a habitable planet', 'the Endurance', 'Dr. Amelia Brand', 

In [ ]:
np_movie = get_noun_phrases(movie_plot)
print (np_movie)

['crop blights', 'dust storms', "humanity's survival", 'Cooper', 'a widowed engineer', 'former NASA pilot', 'farmer', 'his father', 'law', 'Donald', 'teen son', 'Tom', '10-year-old daughter', 'Murph', 'a dust storm', 'patterns', "Murph's dust-covered bedroom floor", 'a ghost', 'them', 'the patterns', 'gravity variations', 'geographic coordinates', 'binary code', 'the coordinates', 'a secret NASA facility', 'Professor John Brand', 'whom', 'Professor Brand', 'a group', 'unknown, presumingly extra-terrestrial beings', 'a wormhole', 'Saturn', 'a path', 'a distant galaxy', 'twelve potentially habitable worlds', 'a supermassive black hole', 'Gargantua', 'Twelve volunteers', 'the wormhole', 'the planets', 'Dr. Mann', 'Laura Miller', 'Wolf Edmunds', 'positive results', 'two plans', 'Plan A', 'an antigravitational propulsion theory', 'settlements', 'space', 'contrast', 'Plan B', 'the Endurance spacecraft', '5,000 frozen human embryos', 'a habitable planet', 'the Endurance', 'Dr. Amelia Brand', 

In [ ]:
print (len(np_movie))

174


In [ ]:
!pip install --quiet sentence-transformers==2.2.0

     |████████████████████████████████| 79 kB 3.3 MB/s 
     |████████████████████████████████| 4.7 MB 11.4 MB/s 
     |████████████████████████████████| 1.3 MB 42.2 MB/s 
     |████████████████████████████████| 120 kB 51.9 MB/s 
     |████████████████████████████████| 6.6 MB 43.7 MB/s 


In [ ]:
from sentence_transformers import SentenceTransformer
model= SentenceTransformer('all-MiniLM-L12-v2')

def get_doc_and_candidate_embeddings(transformermodel,doc_text,candiate_phrases):
  doc_embedding = transformermodel.encode([doc_text])
  candidate_embeddings = transformermodel.encode(candiate_phrases)
  return doc_embedding,candidate_embeddings

/usr/local/lib/python3.7/dist-packages/huggingface_hub/snapshot_download.py:11: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/file_download.py:624: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/573 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/352 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
document_embedd, keyphrase_embedds = get_doc_and_candidate_embeddings(model,movie_plot,np_movie)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Extract similarity within words, and between words and the document
word_doc_similarity = cosine_similarity(keyphrase_embedds, document_embedd)
word_doc_similarity = word_doc_similarity.flatten()
# print (word_doc_similarity)
print (word_doc_similarity.flatten())
print (word_doc_similarity.shape)

keywords_idx = np.argsort(word_doc_similarity)
keywords_idx = keywords_idx[::-1]

print (word_doc_similarity[keywords_idx])



[ 0.18878315  0.30890748  0.19653101  0.4952594   0.24791282  0.43507755
  0.2691992   0.26511556  0.09100178  0.18633679  0.18506664  0.20059702
  0.1311753   0.29839468  0.29365158  0.25755233  0.4212784   0.20368008
  0.0976331   0.2541682   0.12125229  0.1299075   0.26749125  0.16643421
  0.36354142  0.33777606  0.23620746  0.2844899   0.10333371  0.27134493
  0.20259435  0.1422944   0.15861234  0.18034278  0.16667879  0.04951388
  0.12782283  0.11464315  0.23860085  0.20876336  0.26222226  0.19652264
  0.14101204  0.03850467  0.1279912   0.07053571 -0.02896192  0.10336627
  0.26954484  0.10986585  0.02114088  0.15086797  0.09476648  0.1583384
  0.06640968  0.2421687   0.2771048   0.17090417  0.1577236   0.2768715
  0.17621168  0.13530192  0.16743076  0.28684896  0.24468878  0.19277936
  0.16052543  0.21557987  0.26777646  0.05437357  0.15963247  0.15232876
  0.23338315  0.14534836  0.05543496  0.13458715  0.23688516  0.03870858
  0.05096184  0.20776393  0.05264094  0.17954355  0.1

In [ ]:
sorted_keywords =[]


for index in keywords_idx:
  sorted_keywords.append(np_movie[index])


print (sorted_keywords)

['Cooper', "Cooper's ship", 'former NASA pilot', "Murph's dust-covered bedroom floor", 'the dust patterns', "Murph's old bedroom", "Murph's room", 'a secret NASA facility', 'Professor John Brand', 'the younger Murph', 'Murph\'s "ghost', "encode NASA's coordinates", 'dust storms', "John Brand's death", 'an elderly Murph', 'the adult Murph', 'Murph', 'a dust storm', "Mann's planet", 'the crew', 'The crew', 'Professor Brand', "Prof. Brand's daughter", 'a distraught Murph', 'unknown, presumingly extra-terrestrial beings', 'aliens', 'space', 'farmer', 'seven Earth years', 'binary code', 'his father', 'Dr. Mann', 'patterns', 'the patterns', 'a spacefaring civilization', "Murph's wristwatch", 'A deranged Dr. Mann', 'a widowed engineer', "Miller's planet", 'Dr. Amelia Brand', 'the tesseract', 'the wormhole', "Miller's ship wreckage", 'human habitation', 'whom', 'the "ghost', 'a landing craft', 'Mann', 'Earth', 'him', 'Morse code', 'land', 'the mission', 'the planet', "Edmunds' planet", '"Edmun

Thanks to Maximum Marginal Relevance algorithm from Keybert https://maartengr.github.io/KeyBERT/api/mmr.html

In [ ]:
from typing import List, Tuple
import itertools
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def mmr(doc_embedding: np.ndarray,
        word_embeddings: np.ndarray,
        words: List[str],
        top_n: int = 6,
        diversity: float = 0.95) -> List[Tuple[str, float]]:
    """ Calculate Maximal Marginal Relevance (MMR)
    between candidate keywords and the document.


    MMR considers the similarity of keywords/keyphrases with the
    document, along with the similarity of already selected
    keywords and keyphrases. This results in a selection of keywords
    that maximize their within diversity with respect to the document.

    Arguments:
        doc_embedding: The document embeddings
        word_embeddings: The embeddings of the selected candidate keywords/phrases
        words: The selected candidate keywords/keyphrases
        top_n: The number of keywords/keyhprases to return
        diversity: How diverse the select keywords/keyphrases are.
                   Values between 0 and 1 with 0 being not diverse at all
                   and 1 being most diverse.

    Returns:
         List[Tuple[str, float]]: The selected keywords/keyphrases with their distances

    """

    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)

    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # Update keywords & candidates
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [(words[idx], round(float(word_doc_similarity.reshape(1, -1)[0][idx]), 4)) for idx in keywords_idx]

In [ ]:
final_distractors = mmr(document_embedd,keyphrase_embedds,np_movie,5)
print (final_distractors)

[('Cooper', 0.4953), ('5,000 frozen human embryos', 0.0948), ('an antigravitational propulsion theory', -0.029), ('crop blights', 0.1888), ("Murph's dust-covered bedroom floor", 0.4213)]


In [ ]:
final_distractors = mmr(document_embedd,keyphrase_embedds,np_movie,5,diversity=0.25)
print (final_distractors)

[('Cooper', 0.4953), ("Murph's dust-covered bedroom floor", 0.4213), ('former NASA pilot', 0.4351), ('the dust patterns', 0.3966), ('Professor John Brand', 0.3378)]


In [ ]:
from typing import List, Tuple
import itertools
def max_sum_similarity(doc_embedding: np.ndarray,
                       word_embeddings: np.ndarray,
                       words: List[str],
                       top_n: int,
                       nr_candidates: int) -> List[Tuple[str, float]]:
    """ Calculate Max Sum Distance for extraction of keywords
    We take the 2 x top_n most similar words/phrases to the document.
    Then, we take all top_n combinations from the 2 x top_n words and
    extract the combination that are the least similar to each other
    by cosine similarity.
    NOTE:
        This is O(n^2) and therefore not advised if you use a large top_n
    Arguments:
        doc_embedding: The document embeddings
        word_embeddings: The embeddings of the selected candidate keywords/phrases
        words: The selected candidate keywords/keyphrases
        top_n: The number of keywords/keyhprases to return
        nr_candidates: The number of candidates to consider
    Returns:
         List[Tuple[str, float]]: The selected keywords/keyphrases with their distances
    """
    if nr_candidates < top_n:
        raise Exception("Make sure that the number of candidates exceeds the number "
                        "of keywords to return.")

    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, word_embeddings)
    distances_words = cosine_similarity(word_embeddings, word_embeddings)

    # Get 2*top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [words[index] for index in words_idx]
    candidates = distances_words[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = 100_000
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [(words_vals[idx], round(float(distances[0][idx]), 4)) for idx in candidate]




In [ ]:
print ( max_sum_similarity(document_embedd,keyphrase_embedds,np_movie,6,20))

[("Mann's planet", 0.3089), ('an elderly Murph', 0.4351), ('dust storms', 0.2651), ("encode NASA's coordinates", 0.091), ('Professor John Brand', 0.2006), ("Cooper's ship", 0.0976)]


# Find Similar Documents

In [ ]:
df.head()

,Unnamed: 0,Movie Name,Wiki Link,Plot
0,0,White Noise,https://en.wikipedia.org//wiki/White_Noise_(20...,Jonathan Rivers is an architect and lives with...
1,1,Coach Carter,https://en.wikipedia.org//wiki/Coach_Carter,"Ken Carter lives in Richmond, California. He b..."
2,2,Elektra,https://en.wikipedia.org//wiki/Elektra_(2005_f...,"After being killed in Daredevil, Elektra Natch..."
3,3,Racing Stripes,https://en.wikipedia.org//wiki/Racing_Stripes,"During a thunderstorm, a traveling circus, Cir..."
4,4,Tom and Jerry: Blast Off to Mars,https://en.wikipedia.org//wiki/Tom_and_Jerry:_...,Tom (voiced by Bill Kopp) chases Jerry as usua...


In [ ]:
all_plots = df['Plot'].tolist()
print (len(all_plots))

3379


In [ ]:
batmanbegins_df  = df[df['Movie Name'].str.match('Batman Begins')]
batmanbegins_plot = batmanbegins_df.iloc[0]['Plot']
print (batmanbegins_plot)

In Gotham City, a young Bruce Wayne falls down a dry well and is attacked by a swarm of bats, developing a fear of them. Attending the opera with his parents, Thomas and Martha, Bruce becomes frightened by performers masquerading as bats and asks to leave. Outside, mugger Joe Chill murders Bruce's parents in front of him, and the orphaned Bruce is raised by the family butler, Alfred Pennyworth. 14 years later, Chill is paroled after testifying against mafia boss Carmine Falcone. Bruce intends to murder Chill to avenge his parents, but one of Falcone's assassins does so first. Bruce's childhood friend Rachel Dawes berates him for acting outside the justice system, saying that his father would be ashamed. After confronting Falcone, who tells him that real power comes from being feared, Bruce spends the next seven years traveling the world training in combat and immersing himself in the criminal underworld.

In a Bhutan prison, he meets Henri Ducard, who recruits him to the League of Shad

In [ ]:
document_embedd, plot_embedds = get_doc_and_candidate_embeddings(model,batmanbegins_plot,all_plots)

In [ ]:
word_doc_similarity = cosine_similarity(plot_embedds, document_embedd)
word_doc_similarity = word_doc_similarity.flatten()

plots_idx = np.argsort(word_doc_similarity)
plots_idx = plots_idx[::-1]

sorted_plots =[]
sorted_titles=[]

all_titles=df['Movie Name'].tolist()

for index in plots_idx:
  sorted_plots.append(all_plots[index])
  sorted_titles.append(all_titles[index])


N = 6 

for plot,title in zip(sorted_plots[:N],sorted_titles[:N]):
  print (title)
  print (plot)
  print ("\n\n")


Batman Begins
In Gotham City, a young Bruce Wayne falls down a dry well and is attacked by a swarm of bats, developing a fear of them. Attending the opera with his parents, Thomas and Martha, Bruce becomes frightened by performers masquerading as bats and asks to leave. Outside, mugger Joe Chill murders Bruce's parents in front of him, and the orphaned Bruce is raised by the family butler, Alfred Pennyworth. 14 years later, Chill is paroled after testifying against mafia boss Carmine Falcone. Bruce intends to murder Chill to avenge his parents, but one of Falcone's assassins does so first. Bruce's childhood friend Rachel Dawes berates him for acting outside the justice system, saying that his father would be ashamed. After confronting Falcone, who tells him that real power comes from being feared, Bruce spends the next seven years traveling the world training in combat and immersing himself in the criminal underworld.

In a Bhutan prison, he meets Henri Ducard, who recruits him to the 

## Precompute embeddings and store as csv

In [ ]:
from sentence_transformers import SentenceTransformer
model= SentenceTransformer('all-MiniLM-L12-v2')

In [ ]:
movie_plots = df['Plot'].tolist()
print (len(movie_plots))
print (movie_plots[0])

3379
Jonathan Rivers is an architect and lives with his wife Anna until her unexpected death. Eventually, he is contacted by Raymond Price, whose son has also died. He says he has recorded messages from Anna through electronic voice phenomena (EVP). While Jonathan is initially dismissive and angered, he later learns about his wife's tragic drowning. Desperate, he begins to believe that the recorded voice is indeed that of his wife, and becomes obsessed with trying to contact her himself. He is warned by a psychic that, while she takes measures to avoid hostile entities, EVP is an indiscriminate process that offers no such safeguards. A woman named Sarah Tate, who also came to Raymond for his EVP work because she lost her fiancé, befriends Jonathan.

Raymond is found dead. Jonathan begins to be followed by three demons attracted by his obsession with EVP, and finds that some of the messages he receives are from people who are not yet dead, but may soon be. Jonathan hears cries from a wo

In [ ]:
embeddings = model.encode(movie_plots,show_progress_bar=True)

Batches:   0%|          | 0/106 [00:00<?, ?it/s]

In [ ]:
df['full_embeddings']= embeddings.tolist()
df.head(3)

,Unnamed: 0,Movie Name,Wiki Link,Plot,full_embeddings
0,0,White Noise,https://en.wikipedia.org//wiki/White_Noise_(20...,Jonathan Rivers is an architect and lives with...,"[-0.0726638212800026, 0.05636344477534294, 0.0..."
1,1,Coach Carter,https://en.wikipedia.org//wiki/Coach_Carter,"Ken Carter lives in Richmond, California. He b...","[-0.0666189044713974, 0.04147614911198616, 0.0..."
2,2,Elektra,https://en.wikipedia.org//wiki/Elektra_(2005_f...,"After being killed in Daredevil, Elektra Natch...","[-0.03511177748441696, -0.01160984206944704, 0..."


In [ ]:
movie_plots_with_embeddings = "/content/drive/My Drive/Practical Introduction to NLP Course/Module 3/American_Movie_Plots_2005_2021_v1_embeddings.csv"

In [ ]:
df.to_csv(movie_plots_with_embeddings,index=False)